In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import classification_report
import seaborn as sns
from scipy import stats
from pathlib import Path
import pickle
import numpy as np
import os

In [2]:
# display all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [3]:
cd ~/pickle/stoptostop

/home/team9/pickle/stoptostop


In [4]:
# https://zhuanlan.zhihu.com/p/98124110
def getfiles():
    filenames=os.listdir(r'/home/team9/tmp/data/leavetimes')
    print(filenames)

### Second way to get journey time: add the infomation of starting stop, calculatr the journey time for each stop

In [5]:
'''
This function is to add columns for the beginning to one exact stops model: eg. first to second stop, first to fifth stop 
would prefer this function, reasons:
1. much higher accuracy
2. easy to use for backend, get two stopid and then minus
'''

'\nThis function is to add columns for the beginning to one exact stops model: eg. first to second stop, first to fifth stop \nwould prefer this function, reasons:\n1. much higher accuracy\n2. easy to use for backend, get two stopid and then minus\n'

In [6]:
# delete the first rows and it related rows that there's not starting with 1
def add_columns(x):
    # drop useless columns
    x = x.drop(columns=['Unnamed: 0','VEHICLEID'])
    x = x.sort_values(by=['DAYOFSERVICE','TRIPID','PROGRNUMBER'])
    x = x.reset_index(drop=True)
    x["DAYOFSERVICE"] =  x["DAYOFSERVICE"].astype("datetime64[ns]")
    x['LINEID'] = x['LINEID'].astype('object')
    # delete the first duplicate programnum everyday, set its start time for each line of this tripid
    first_rows = x.drop_duplicates(subset=['TRIPID','DAYOFSERVICE'], keep='first')
    first_rows['START_TIME'] = first_rows['ACTUALTIME_DEP']
    first_rows['PLANNED_START_TIME'] = first_rows['PLANNEDTIME_DEP']
    first_rows['flag'] = 0
    first_rows['flag'].loc[first_rows['PROGRNUMBER'] == 1] = 1
    first_rows = first_rows[['DAYOFSERVICE','TRIPID','LINEID','START_TIME','PLANNED_START_TIME','flag']]
    x = pd.merge(x, first_rows,on=['DAYOFSERVICE','TRIPID','LINEID'], how='left')
    x = x.loc[x['flag'] == 1]
    # calculate the journey time 
    x['JOURNEY_TIME'] = x['ACTUALTIME_ARR'] - x['START_TIME']
    x['PLANNED_JOURNEY_TIME'] = x['PLANNEDTIME_ARR'] - x['PLANNED_START_TIME']
    x = x.drop(columns=['flag'])
    x = x.reset_index(drop=True)
    return x

## add direction with trip.csv
because the total leavetimes is too huge so didn't conclude direction while spplit the file  

In [7]:
trips = pd.read_csv(f'/home/team9/tmp/data/rt_trips_DB_2018.txt', sep=";", parse_dates=[0],usecols=['DAYOFSERVICE', 'TRIPID', 'LINEID','DIRECTION'])

In [8]:
trips.dtypes

DAYOFSERVICE    datetime64[ns]
TRIPID                   int64
LINEID                  object
DIRECTION                int64
dtype: object

In [9]:
trips

,DAYOFSERVICE,TRIPID,LINEID,DIRECTION
0,2018-02-07,6253783,68,1
1,2018-02-07,6262138,25B,2
2,2018-02-07,6254942,45A,2
3,2018-02-07,6259460,25A,1
4,2018-02-07,6253175,14,1
...,...,...,...,...
2182632,2018-05-14,6765849,123,2
2182633,2018-05-14,6765469,75,1
2182634,2018-05-14,6765486,33D,2
2182635,2018-05-14,6764987,70,1


## check outliers and delete rows
the second way without stop interval columns

In [10]:
def remove_rows(x):
    # remove rows with negative journey time
    x =  x.drop(index = x[(x.JOURNEY_TIME <0)].index.tolist())
    x =  x.drop(index = x[(x.PLANNED_JOURNEY_TIME <=0)].index.tolist())
    # remove null values although didn't exist in test set, sometimes line don't have directions
    x.dropna(axis=1, how='all', inplace=True)
    x.dropna(axis=0, how='any', inplace=True)
    # remove stop interval != 1 which is not continuous
    #x = x.drop(index = x[(x.STOP_INTERVAL != 1)].index.tolist())
    x = x.reset_index(drop=True)
    return x

## add month, hour, dayofweek

In [11]:
def add_date(x):
#     x["DAYOFSERVICE"] =  x["DAYOFSERVICE"].astype("datetime64[ns]") 
    x['MONTH'] = x['DAYOFSERVICE'].dt.month
    x['DAYOFWEEK'] = x['DAYOFSERVICE'].dt.weekday
    x['HOUR'] = x['ACTUALTIME_ARR'].apply(lambda row: row//3600)
    # if 24 then change to midnight and increase day by 1
    x['DAYOFSERVICE'].loc[x['HOUR'] >= 24] = x['DAYOFSERVICE']+ dt.timedelta(days=1)
    x['HOUR'].loc[x['HOUR'] >= 24] -= 24
    x = x.drop(columns=['TRIPID','PLANNEDTIME_ARR', 'PLANNEDTIME_DEP', 'ACTUALTIME_ARR', 'ACTUALTIME_DEP'])
    return x

## clean the weather

In [12]:
weather = pd.read_csv(f'/home/team9/tmp/data/Custom_location_53_345035_-6_267261_62b5c8e6c91d98000ba01ceb.csv', parse_dates=[1])

In [13]:
#weather.dtypes

In [14]:
def date_from_epoch(x):
    y = dt.datetime.fromtimestamp (x).strftime('%Y-%m-%d')
    return y   

# weather['DAYOFSERVICE'] = weather['dt'].apply(lambda x: date_from_epoch(x))

In [15]:
def round_epoch_hour(x):
    day = x // 86400
    day = day * 86400
    time = x - day
    round_time = time // 3600
    return round_time
# weather['HOUR'] = weather['dt'].apply(lambda x: round_epoch_hour(x))

In [16]:
def clean_weather(x):
    # x = x[['dt','temp','weather_main']]
    x = x.drop(columns=['dt_iso','timezone','city_name','lat','lon','sea_level','grnd_level','wind_gust','rain_1h','rain_3h','snow_1h','snow_3h','weather_description','weather_icon'])
    x['DAYOFSERVICE'] = x['dt'].apply(lambda x: date_from_epoch(x))
    x["DAYOFSERVICE"] =  x["DAYOFSERVICE"].astype("datetime64[ns]") 
    x['HOUR'] = x['dt'].apply(lambda x: round_epoch_hour(x))
    return x

In [17]:
weather = clean_weather(weather)

## merge the leavetimes with weather

## encode time

In [18]:
def cal_rush(df):
    df['rush_hour'] = 0
    rush_hour = [7,8,9,16,17,18,19]
    weekends = [5,6]
    for i in rush_hour:
        df['rush_hour'].loc[df['HOUR'] == i] = 1
    for j in weekends:
        df['rush_hour'].loc[df['DAYOFWEEK'] == j] = 0
    return df

### choose input for models

In [19]:
def choose_input(df):
    df['weather_main'] = df['weather_main'].apply(lambda x: 1 if (x == "Rain" or x == "Mist" or x == "Thunderstorm") else 0)
    df = df.drop({'DAYOFSERVICE', 'LINEID', 'START_TIME','PLANNED_START_TIME','PLANNED_JOURNEY_TIME','dt','visibility','dew_point','feels_like','temp_min','temp_max','pressure','humidity','wind_deg','clouds_all','weather_id','wind_speed','temp'}, axis=1)
    numbers = [feature for feature in list(df.columns)]
    for feature in numbers:
        if feature != 'weather_main':
            df[feature] = df[feature].astype('int32')
        if feature == 'weather_main':
            df[feature] = df[feature].astype('category')
    df.dropna(axis=0, how='any', inplace=True)
    return df

### Random Forest

In [29]:
def random_forest(dt):
    y=dt["JOURNEY_TIME"] 
    x=dt.drop('JOURNEY_TIME', axis=1)
    features=x.columns.values.tolist()
    
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)
    rfc = RandomForestRegressor(n_estimators=20, n_jobs=-1, max_features=10, max_depth=34, min_samples_leaf=10, min_samples_split=15)
    rfc.fit(X_train, y_train)

    #add to avg size of train/test arrays
    X_train_size_avg = X_train.shape[0]
    X_test_size_avg = X_test.shape[0]
    prediction = rfc.predict(X_test)
    #modelEvalucation(y_test, prediction)
    MAPEs = metrics.mean_absolute_percentage_error(y_test, prediction)
    R2 = metrics.r2_score(y_test, prediction)
    return [rfc, X_train_size_avg, X_test_size_avg, MAPEs, R2]


In [30]:
def get_files(trips,weather):
    X_train_size = []
    X_test_size = []
    MAPEs = []
    R2 = []
    #filenames=os.listdir(r'/home/team9/tmp/data/leavetimes')
    for filepath,dirnames,filenames in os.walk(r'/home/team9/tmp/data/leavetimes'):
        for filename in filenames:
            fullname = os.path.join(filepath, filename)
            print(fullname)
            df = pd.read_csv(fullname, sep=",")
            df = add_columns(df)
            #trips = pd.read_csv(f'/home/team9/tmp/data/rt_trips_DB_2018.txt', sep=";", parse_dates=[0],usecols=['DAYOFSERVICE', 'TRIPID', 'LINEID','DIRECTION'])
            df = pd.merge(df,trips,on=['DAYOFSERVICE','TRIPID','LINEID'],how='left')
            df = remove_rows(df)
            df = add_date(df)
            #weather = pd.read_csv(f'/home/team9/tmp/data/Custom_location_53_345035_-6_267261_62b5c8e6c91d98000ba01ceb.csv', parse_dates=[1])
            #weather = clean_weather(weather)
            df = pd.merge(df, weather, on=['DAYOFSERVICE', 'HOUR'], how='left')
            df.dropna(axis=0, how='any', inplace=True)
            df = cal_rush(df)
            df = choose_input(df)
            rfc,a,b,c,d = random_forest(df)
            X_train_size.append(a)
            X_test_size.append(b)
            MAPEs.append(c)
            R2.append(d)
            #save model
            t=filename+'.pickle'
            f = open(t,'wb')
            pickle.dump(rfc,f, protocol=pickle.HIGHEST_PROTOCOL)
            f.close()
    return X_train_size, X_test_size, MAPEs, R2

In [31]:
X_train_size, X_test_size, MAPEs, R2 = get_files(trips,weather)

/home/team9/tmp/data/leavetimes/leavetimes_65B.csv


/tmp/ipykernel_32238/2240808050.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_rows['START_TIME'] = first_rows['ACTUALTIME_DEP']
/tmp/ipykernel_32238/2240808050.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_rows['PLANNED_START_TIME'] = first_rows['PLANNEDTIME_DEP']
/tmp/ipykernel_32238/2240808050.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

/home/team9/tmp/data/leavetimes/leavetimes_32X.csv


/tmp/ipykernel_32238/2240808050.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_rows['START_TIME'] = first_rows['ACTUALTIME_DEP']
/tmp/ipykernel_32238/2240808050.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_rows['PLANNED_START_TIME'] = first_rows['PLANNEDTIME_DEP']
/tmp/ipykernel_32238/2240808050.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

/home/team9/tmp/data/leavetimes/leavetimes_70.csv


...

/tmp/ipykernel_32238/2240808050.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_rows['START_TIME'] = first_rows['ACTUALTIME_DEP']
/tmp/ipykernel_32238/2240808050.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_rows['PLANNED_START_TIME'] = first_rows['PLANNEDTIME_DEP']
/tmp/ipykernel_32238/2240808050.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

/home/team9/tmp/data/leavetimes/leavetimes_51X.csv


/tmp/ipykernel_32238/2240808050.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_rows['START_TIME'] = first_rows['ACTUALTIME_DEP']
/tmp/ipykernel_32238/2240808050.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_rows['PLANNED_START_TIME'] = first_rows['PLANNEDTIME_DEP']
/tmp/ipykernel_32238/2240808050.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

/home/team9/tmp/data/leavetimes/leavetimes_51D.csv


/tmp/ipykernel_32238/2240808050.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_rows['START_TIME'] = first_rows['ACTUALTIME_DEP']
/tmp/ipykernel_32238/2240808050.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_rows['PLANNED_START_TIME'] = first_rows['PLANNEDTIME_DEP']
/tmp/ipykernel_32238/2240808050.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

In [32]:
average_train = sum(X_train_size)/len(X_train_size)
print(average_train)

593357.6923076923


In [33]:
average_test = sum( X_test_size)/len( X_test_size)
print(average_test)

254296.85384615386


In [34]:
average_MAPE = sum(MAPEs)/len(MAPEs)
print(average_MAPE)

0.12539680863998576


In [35]:
average_R2 = sum(R2)/len(R2)
print(average_R2)

0.9379264177170826


593357.6923076923  
254296.85384615386  
0.13814975527674683  
0.9235674094058496

In [40]:
test = pickle.load(open("./leavetimes_65B.csv.pickle",'rb'))

In [41]:
test.predict(X_test)

NameError: name 'X_test' is not defined